In [1]:
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\szige\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\szige\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
directory = 'flickr30k_images'
images_directory = 'flickr30k_images'
captions_directory = 'results.csv'
df = pd.read_csv(os.path.join(directory, captions_directory), delimiter='|', on_bad_lines='skip')
df.rename(columns={' comment': 'captions', ' image_name': 'image_name'}, inplace=True)
df

,image_name,comment_number,captions
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .
...,...,...,...
158910,998845445.jpg,0,A man in shorts and a Hawaiian shirt leans ov...
158911,998845445.jpg,1,"A young man hanging over the side of a boat ,..."
158912,998845445.jpg,2,A man is leaning off of the side of a blue an...
158913,998845445.jpg,3,"A man riding a small boat in a harbor , with ..."


In [3]:
stop_words = set(stopwords.words('english'))

In [6]:
def clean_caption(row):
    row['captions'] = str(row['captions'])
    row['captions'] = row['captions'].lower()
    row['captions'] = ''.join([i for i in row['captions'] if not i.isdigit() and not i in string.punctuation])
    word_tokens = word_tokenize(row['captions'])
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence.insert(0,'<start>')
    filtered_sentence.append('<end>')
    row['captions'] = ' '.join(filtered_sentence)

In [7]:
df.apply(clean_caption, axis=1)
df

,image_name,comment_number,captions
0,1000092795.jpg,0,<start> two young guys shaggy hair look hands ...
1,1000092795.jpg,1,<start> two young white males outside near man...
2,1000092795.jpg,2,<start> two men green shirts standing yard <end>
3,1000092795.jpg,3,<start> man blue shirt standing garden <end>
4,1000092795.jpg,4,<start> two friends enjoy time spent together ...
...,...,...,...
158910,998845445.jpg,0,<start> man shorts hawaiian shirt leans rail p...
158911,998845445.jpg,1,<start> young man hanging side boat like fog r...
158912,998845445.jpg,2,<start> man leaning side blue white boat sits ...
158913,998845445.jpg,3,<start> man riding small boat harbor fog mount...


In [8]:
df = df.groupby('image_name').agg({'captions': lambda x: list(x)}).reset_index()

In [11]:
df

,image_name,captions
0,1000092795.jpg,[<start> two young guys shaggy hair look hands...
1,10002456.jpg,[<start> several men hard hats operating giant...
2,1000268201.jpg,[<start> child pink dress climbing set stairs ...
3,1000344755.jpg,[<start> someone blue shirt hat standing stair...
4,1000366164.jpg,[<start> two men one gray shirt one black shir...
...,...,...
31778,997338199.jpg,[<start> woman writing pad room gold decorated...
31779,997722733.jpg,[<start> person red shirt climbing rock face c...
31780,997876722.jpg,[<start> two male construction workers working...
31781,99804383.jpg,[<start> older busker glasses plays eastern st...


In [12]:
df.to_csv('captions.csv')